In [1]:
import pandas as pd
from tqdm import tqdm

import warnings 
warnings.filterwarnings(action='ignore')

In [2]:
df = pd.read_csv('../../dataset/total_baseball_PCODE&배럴타구추가.csv')
df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(str(x)))
df.head()

,NAME,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,장타,OPS,투구,avLI,RE24,WPA,year,Date,PCODE,barrel
0,정훈,429,한화,W,4:3,7,3B,1,2,0,...,0.000,0.000,8,1.51,-0.72,-0.046,2018,2018-04-29,60523.0,0.0
1,정훈,502,KIA,L,6:12,6,1B,1,4,0,...,0.250,0.361,12,1.09,-0.33,-0.032,2018,2018-05-02,60523.0,0.0
2,정훈,504,@SK,L,4:5,9,PH,0,1,1,...,0.800,1.073,3,0.44,1.00,0.025,2018,2018-05-04,60523.0,1.0
3,정훈,508,@LG,W,4:2,5,PH,0,1,0,...,0.750,1.058,6,0.49,-0.26,-0.013,2018,2018-05-08,60523.0,0.0
4,정훈,510,@LG,W,7:2,9,PH,0,1,0,...,0.667,0.979,3,0.02,-0.26,-0.001,2018,2018-05-10,60523.0,1.0


In [3]:
def opposing_team(df):
    df1 = df.reset_index().groupby(['index', 'NAME', "Date", "상대"]).size().unstack().reset_index().fillna(0).set_index('index')

    df1['원정경기수'] = df1["@KIA"] + df1["@KT"] + df1['@LG'] + df1['@NC'] + df1['@SK'] + \
                    df1['@SSG'] + df1['@넥센'] + df1['@두산'] + df1['@롯데'] + \
                    df1['@삼성'] + df1['@키움'] + df1['@한화']
    df1['홈경기수'] = df1["KIA"] + df1["KT"] + df1['LG'] + df1['NC'] + df1['SK'] + \
                    df1['SSG'] + df1['넥센'] + df1['두산'] + df1['롯데'] + \
                    df1['삼성'] + df1['키움'] + df1['한화']
    df1['LG'] = df1['LG'] + df1["@LG"]
    df1['KIA'] = df1['KIA'] + df1["@KIA"]
    df1['KT'] = df1['KT'] + df1["@KIA"]
    df1['키움'] = df1['키움'] + df1["@키움"] + df1['넥센'] + df1['@넥센']
    df1['두산'] = df1['두산'] + df1["@두산"]
    df1['한화'] = df1['한화'] + df1["@한화"]
    df1['NC'] = df1['NC'] + df1["@NC"]
    df1['롯데'] = df1['롯데'] + df1["@롯데"]
    df1['삼성'] = df1['삼성'] + df1["@삼성"]
    df1['SSG'] = df1['SSG'] + df1['@SSG'] + df1['@SK'] + df1["SK"]
    df1 = df1[['NAME', 'Date', 'LG', 'KIA', 'KT', '키움', '두산', '한화', 'NC', '롯데', '삼성', 'SSG', '홈경기수', '원정경기수']]
    
    return df.join(df1.iloc[:, 2:])

In [4]:
df1 = opposing_team(df)
df1.head()

,NAME,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,KT,키움,두산,한화,NC,롯데,삼성,SSG,홈경기수,원정경기수
0,정훈,429,한화,W,4:3,7,3B,1,2,0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1,정훈,502,KIA,L,6:12,6,1B,1,4,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,정훈,504,@SK,L,4:5,9,PH,0,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
3,정훈,508,@LG,W,4:2,5,PH,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,정훈,510,@LG,W,7:2,9,PH,0,1,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [5]:
df1.columns

Index(['NAME', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타',
       '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타',
       '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA', 'year',
       'Date', 'PCODE', 'barrel', 'LG', 'KIA', 'KT', '키움', '두산', '한화', 'NC',
       '롯데', '삼성', 'SSG', '홈경기수', '원정경기수'],
      dtype='object')

In [6]:
def make_train_set(df, day):
    columns = ['NAME', 'PCODE', 'Date', '선발', '타수', '득점', '안타', '2타', '3타', '홈런',
                '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진',
                '병살', '희타', '희비', '투구', 'barrel', '타율',
                'LG', 'KIA', 'KT', '키움', '두산', '한화', 'NC', '롯데', '삼성', 
                'SSG', '홈경기수', '원정경기수', 
                '장타', '출루', 'OPS']

    name_list = df['NAME'].unique()
    year_list = [2018,2019,2020,2021]
    df['Date'] = df['Date'].apply(lambda x: pd.to_datetime(str(x)))

    df_list = []
    for name in tqdm(name_list):
        for year in year_list:
            df_tmp = df[(df['NAME'] == name)&(df['Date'].dt.year == year)]
            if len(df_tmp) ==0 : continue
            if (year == 2021) & (df_tmp["Date"].iloc[0] != '2021-04-03'):
                df_tmp.loc[-1] = 0
                df_tmp.loc[-1, 'Date'] = pd.to_datetime('2021-04-03')
                df_tmp.index = df_tmp.index + 1
                df_tmp.sort_index(inplace=True)
            pcode = int(df_tmp['PCODE'].iloc[1])
            df_tmp = df_tmp.set_index('Date').resample(f'{day}D').sum().reset_index()
            df_tmp = df_tmp[df_tmp['타수'] != 0]
            times = [(df_tmp["Date"].iloc[i+1] - df_tmp['Date'].iloc[i]).days for i in range(len(df_tmp["Date"])-1)] + [0]
            df_tmp['times'] = times
            df_tmp['NAME'] = name
            df_tmp['PCODE'] = pcode
            df_tmp['타율'] = df_tmp['안타'] / df_tmp['타수']
            df_tmp['장타'] = df_tmp['루타'] / df_tmp['타수']
            df_tmp['출루'] = (df_tmp['안타'] + df_tmp['볼넷'] + df_tmp['사구']) / (df_tmp['타수'] + df_tmp['볼넷'] + df_tmp['사구'] + df_tmp['희비'])
            df_tmp['OPS'] = df_tmp['출루'] + df_tmp['장타']
            

            df_tmp = df_tmp[(df_tmp['times'] == day) | (df_tmp['times'] == 0)]
            df_tmp['OPS'] = list(df_tmp['OPS'].iloc[1:]) + [None]
            df_tmp['장타'] = list(df_tmp['장타'].iloc[1:]) + [None]
            df_tmp['출루'] = list(df_tmp['출루'].iloc[1:]) + [None]
            df_tmp = df_tmp[df_tmp['OPS'].notnull()]

            df_list.append(df_tmp)

    return pd.concat(df_list)[columns].sort_values(['NAME', 'Date']).reset_index(drop=True)


In [7]:
def make_test_set(df):
    columns = ['NAME', 'PCODE', '선발', '타수', '득점', '안타', '2타', '3타', '홈런',
                '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진',
                '병살', '희타', '희비', '투구', 'barrel', '타율', 
                'LG', 'KIA', 'KT', '키움', '두산', '한화', 'NC', '롯데', '삼성', 
                'SSG', '홈경기수', '원정경기수']

    pred_player_list = df[df['PCODE'].apply(lambda x : x in \
        [76232, 68050, 75847, 67341, 79192,78224, 78513, 76290, 79215, 67872])]['NAME'].unique()
    
    df_tmp = df[(df['Date'] >= '2021-08-02') & df["NAME"].apply(lambda x : x in pred_player_list)]
    df_tmp = df_tmp.groupby(["NAME", "PCODE"]).sum().reset_index()

    df_tmp['타율'] = df_tmp['안타'] / df_tmp['타수']

    df_final = df_tmp[columns]

    return df_final

In [8]:
train_df = make_train_set(df1, 24)
train_df.head()

100%|██████████| 327/327 [00:24<00:00, 13.42it/s]


,NAME,PCODE,Date,선발,타수,득점,안타,2타,3타,홈런,...,한화,NC,롯데,삼성,SSG,홈경기수,원정경기수,장타,출루,OPS
0,LT김민수,67504,2021-04-03,0,6,1,1,0,0,0,...,0.0,2.0,0.0,0.0,0.0,1.0,3.0,0.266667,0.250000,0.516667
1,LT김민수,67504,2021-04-27,4,15,2,3,1,0,0,...,3.0,0.0,0.0,1.0,0.0,3.0,3.0,0.440678,0.359375,0.800053
2,LT김민수,67504,2021-05-21,14,59,5,18,5,0,1,...,0.0,3.0,0.0,0.0,0.0,9.0,8.0,0.361702,0.314815,0.676517
3,LT김민수,67504,2021-06-14,12,47,7,11,1,1,1,...,4.0,3.0,0.0,3.0,3.0,6.0,10.0,0.235294,0.333333,0.568627
4,SK김재현,76869,2018-05-15,5,15,6,3,0,0,0,...,3.0,0.0,0.0,3.0,0.0,11.0,6.0,1.500000,0.500000,2.000000


In [10]:
train_df.columns

Index(['NAME', 'PCODE', 'Date', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타',
       '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '투구',
       'barrel', '타율', 'LG', 'KIA', 'KT', '키움', '두산', '한화', 'NC', '롯데', '삼성',
       'SSG', '홈경기수', '원정경기수', '장타', '출루', 'OPS'],
      dtype='object')

In [11]:
test_df = make_test_set(df1)
test_df

,NAME,PCODE,선발,타수,득점,안타,2타,3타,홈런,루타,...,KT,키움,두산,한화,NC,롯데,삼성,SSG,홈경기수,원정경기수
0,강백호,68050.0,21,80,10,27,12,0,3,48,...,0.0,4.0,0.0,1.0,0.0,3.0,5.0,3.0,12.0,10.0
1,김재환,78224.0,23,80,10,22,4,0,2,32,...,0.0,4.0,0.0,4.0,3.0,2.0,4.0,2.0,9.0,14.0
2,김현수,76290.0,20,68,6,18,4,1,2,30,...,2.0,2.0,0.0,0.0,4.0,3.0,3.0,3.0,13.0,7.0
3,로맥,67872.0,9,36,5,7,1,0,1,11,...,0.0,1.0,2.0,0.0,4.0,0.0,1.0,0.0,10.0,4.0
4,박건우,79215.0,23,83,16,25,7,0,1,35,...,0.0,4.0,0.0,4.0,3.0,2.0,4.0,2.0,9.0,14.0
5,양의지,76232.0,21,77,11,26,5,0,3,40,...,0.0,0.0,3.0,7.0,0.0,4.0,0.0,3.0,11.0,10.0
6,이정후,67341.0,3,14,0,6,1,0,0,7,...,2.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0
7,전준우,78513.0,21,84,11,19,5,0,1,27,...,5.0,2.0,2.0,2.0,5.0,0.0,2.0,0.0,10.0,12.0
8,채은성,79192.0,2,8,1,1,1,0,0,2,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0
9,최정,75847.0,21,67,13,16,2,1,6,38,...,0.0,2.0,2.0,0.0,4.0,0.0,3.0,0.0,12.0,10.0
